In [2]:
import pandas as pd

from config import Config

important_filed = ['text', 'content-desc', 'resource-id', 'activity']
config = Config()


# %%

def remove_oracles(events):
    return events[events['event_type'] != 'oracle'].reset_index()


def remove_unimportant_columns(events):
    return events[['class', 'content-desc', 'text', 'activity', 'resource-id']]


def add_type(events, type):
    events.rename(columns={'content-desc': 'content_desc', 'resource-id': 'id'}, inplace=True)
    for col in events.columns:
        events.rename(columns={col: type + '_' + col}, inplace=True)
    return events


def clean_test(events, type):
    events = remove_oracles(events)
    events = remove_unimportant_columns(events)
    events = add_type(events, type)
    return events


def clean_df(df):
    return df


def get_mig_events(series, type, subjects):
    app_name = series[type + '_app']
    events = pd.read_json(subjects + '_gt/' + app_name + '.json').fillna('')
    events = clean_test(events, type)
    events[type + '_app'] = app_name.replace('-', '')
    return events


# %%

craft_map = pd.read_csv('craft_map.csv')
s_list = []
for i, series in craft_map.iterrows():
    src_events = get_mig_events(series, 'src', 'craft')
    target_events = get_mig_events(series, 'target', 'craft')
    src_index = series['src_index']
    target_index = series['target_index']
    src_event = src_events.iloc[src_index]
    target_event = target_events.iloc[target_index]
    row = pd.concat([src_event, target_event], axis=0)
    s_list.append(row)

df = pd.concat(s_list, axis=1).T.reindex()
df = clean_df(df)
craft_events = df.reset_index(drop=True)

# %%
def get_mig_events_atm(series, type):
    app_name = series['src_app'] + '-' +series['target_app']
    events = pd.read_json('atm_gt/' + app_name + '.json').fillna('')
    events = clean_test(events, type)
    events[type + '_app'] = app_name.replace('-','')
    return events

atm_map = pd.read_csv('atm_map.csv')

s_list = []
for i, series in atm_map.iterrows():
    src_events = get_mig_events(series, 'src', 'atm')
    target_events = get_mig_events_atm(series, 'target')
    src_index = series['src_index']
    target_index = series['target_index']
    src_event = src_events.iloc[src_index]
    target_event = target_events.iloc[target_index]
    row = pd.concat([src_event, target_event], axis=0)
    s_list.append(row)

df = pd.concat(s_list, axis=1).T.reindex()
df = clean_df(df)
atm_events = df.reset_index(drop=True)
total_gt = pd.concat([craft_events, atm_events])
total_gt.to_csv(config.ground_truth, index=False)